# Landcover Classification 

## Authored by: Blake Marshall, Sean Farmer, Jacob Sellers, & Isauro Ramos

The Landcover Classification project aims to... <CONTINUE DESCRIPTION>

#Setup

##Installs

In [ ]:
%pip install gdal
%pip install rasterio
%pip install raster2xyz
%pip install matplotlib
%pip install scikit-learn
%pip install seaborn
%pip install tensorflow

##Imports

In [ ]:
import rasterio
from rasterio.plot import show
from raster2xyz.raster2xyz import Raster2xyz
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from tensorflow.keras import layers, models
import tensorflow as tf


### File Paths

In [ ]:
# I (Sam) downloaded the sample data to my own drive to access it. It will probably be a good idea to make it downloadable
path_samples = 'Data/Images/'
file_name = '1_ang20231028t101421_014_L2A_OE_main_27577724_RFL_ORT.tif'
path_test_tiff = path_samples + file_name


###Rasterio into Dataset?


In [ ]:
dataset_1 = rasterio.open(path_test_tiff)

In [ ]:
# Number of Bands
print(dataset_1.count)
# Image Resolution
print(dataset_1.height, dataset_1.width)
# CRS (Coordinate Reference System)
print(dataset_1.crs)

#### To Pandas DataFrame

In [ ]:
# Open the GeoTIFF file
with rasterio.open(path_test_tiff) as src:
    # Read the data as a numpy array
    data = src.read(1) # Read the first band

    # Create a DataFrame with the pixel values
    df = pd.DataFrame(data)

In [ ]:
# this is the first band of the tiff in relationship to the pixels. 10x10 size
df

In [ ]:
data[0,1]

#### To Numpy Array

In [ ]:
# Open the GeoTIFF file
with rasterio.open(path_test_tiff) as dataset:
    # Read the data as a numpy array
    data_3D = dataset.read()

    # Print the data
    print(data_3D.shape)

    # Visualize the data
    show(dataset)

In [ ]:
with rasterio.open(path_test_tiff) as dataset:
    # Read the bands as numpy arrays
    band_57 = dataset.read(57)  # Adjust with correct band number
    band_35 = dataset.read(35)  # Adjust with correct band number
    band_21 = dataset.read(21)  # Adjust with correct band number

    # Stack the bands to form a 3-channel (RGB) image
    rgb_image = np.stack([band_57, band_35, band_21], axis=-1)
    # Normalize the values (optional, depending on the image data scale)
    rgb_image = rgb_image / np.max(rgb_image)  # Normalize to [0, 1]

    # Plot the image
    plt.imshow(rgb_image)
    plt.title("Composite Image with Bands 57, 35, 21")
    plt.axis('off')  # Turn off axis
    plt.show()

###Functions using rasterio to make 1D arrays of the bands corrisponding to their pixel.

In [ ]:
def tiff_to_arr(filepath):
  '''
  Description:
    This function takes a filepath to a .tiff file, opens it, and reads it as a
    numpy arr. Then returns said array.
  Input:
    filepath  : The file path to the .tiff file, starting from /content/...
  Output:
    data_3D   : A 3 dimensional array of frequency bands for the pixels of an
                image.
  '''
  with rasterio.open(filepath) as dataset:
      # Read the data as a numpy array
      data_3D = dataset.read()
  return data_3D

In [ ]:
def convert_3D_to_1D(data_3D):
  '''
  Description:
    This function takes a 3 dimensional array of frequence bands when each
    individual frequence reading is a NxN 2D array. So this 3D array is BxNxN
    where B is the number of frequence bands. This function will return a N*NxB
    array. Where every individual frequence corresponding to a pixel is in the
    returned 1D array for each of the N*N pixels.
  Input:
    data_3D         : Numpy Array with 3 dimensions of shape (num_band, num_row, num_col)
  Output:
    bands_per_pixel : Numpy Array of shape (num_row * numcol, num_band)
  '''
  temp_list_1D_arr = []

  # Access the depth (third dimension) and create 1D arrays
  for i in range(data_3D.shape[1]):                 # 10 for both data_3D.shape[1] & data_3D.shape[2] to make the 10x10
    for j in range(data_3D.shape[2]):
      data_1D = data_3D[:, i, j].flatten()          # EX. this will take the [0,0] for every bands then flatten that into a 1D array. For all the bands corresponding to pixel [0,0]
      temp_list_1D_arr.append(data_1D)              # append to the temp list
  bands_per_pixel = np.array(temp_list_1D_arr)      # convert the list to a numpy array... because I want to.
  return bands_per_pixel

In [ ]:
def get_filenames(directory_path):
    '''
     * Description:
     *   gets the name of both files and directories at path_samples
     *   sorted by the first numeric prefix in the filename
     * Input(s):
     *   directory_path: the path to the directory containing sample files
     * Output(s):
     *   Sorted Numpy Array of Filenames, array of strings
    '''
    filenames = []
    for f in listdir(directory_path):
        # Ignore hidden files and directories (like .DS_Store)
        if f.startswith(".") or not isfile(join(directory_path, f)):
            continue

        try:
            # Try to parse the first part of the filename as an integer
            int(f.split("_")[0])
            filenames.append(f)  # Only add to the list if parsing succeeds
        except ValueError:
            print(f"Non-numeric prefix found in filename: {f}")  # Print any problematic filename

    # Sort the valid filenames and convert to a numpy array
    return np.array(sorted(filenames, key=lambda x: int(x.split("_")[0])))

In [ ]:
def make_pandas_dataframe(dir_path, filename, col_labels):
  #ds = convert_3D_to_1D(tiff_to_arr(join(dir_path, filename)))
  ds = tiff_to_arr(join(dir_path, filename))
  df = pd.DataFrame(ds, columns=col_labels)
  df['File'] = filename
  return df

In [ ]:
def get_all_data(sample_directory_path):
  #Creates the frequency labels
  columns_of_frequencies = []
  for i in range(0,373,1):
    columns_of_frequencies.append("frq" + str(i))

  # get an array of the sample file names
  filenames = get_filenames(sample_directory_path)

  ## This is where we would trim the filenames for the ones we want

  #loop through and add to pandas dataframe
  list_df = []
  for i in range(0, len(filenames)):
    list_df.append(make_pandas_dataframe(sample_directory_path, filenames[i], columns_of_frequencies))
    #print(i)

  return pd.concat(list_df)

## PCA

In [ ]:
def pca_make_pandas_dataframe(dir_path, filename, col_labels):
    ds = tiff_to_arr(join(dir_path, filename))  # shape is (373, 10, 10)

    # Reshape to have each pixel position with 373 band values as a row
    reshaped_ds = ds.reshape(ds.shape[0], -1).T  # Shape becomes (100, 373) or (121, 373) depending on the data
    print(f"reshaped_ds shape: {reshaped_ds.shape}")  # Check shape of reshaped_ds

    # Create the DataFrame with band columns
    df = pd.DataFrame(reshaped_ds, columns=col_labels[:reshaped_ds.shape[1]])

    # Dynamically generate X and Y coordinates based on the number of rows in reshaped_ds
    num_rows = reshaped_ds.shape[0]
    x_vals = np.tile(np.arange(10), num_rows // 10)
    y_vals = np.repeat(np.arange(10), num_rows // 10)

    # Adjust lengths in case of rounding issues
    if len(x_vals) != num_rows:
        x_vals = np.resize(x_vals, num_rows)
    if len(y_vals) != num_rows:
        y_vals = np.resize(y_vals, num_rows)

    # Check if lengths match
    print(f"Length of X: {len(x_vals)}, Length of Y: {len(y_vals)}, Num Rows: {num_rows}")
    
    df['X'] = x_vals  # X coordinates
    df['Y'] = y_vals  # Y coordinates

    # Add filename for tracking
    df['File'] = filename

    return df

In [ ]:
def pca_get_all_data(sample_directory_path):
  #Creates the frequency labels
  columns_of_frequencies = []
  for i in range(0,373,1):
    columns_of_frequencies.append("frq" + str(i))

  # get an array of the sample file names
  filenames = get_filenames(sample_directory_path)

  ## This is where we would trim the filenames for the ones we want

  #loop through and add to pandas dataframe
  list_df = []
  for i in range(0, len(filenames)):
    list_df.append(pca_make_pandas_dataframe(sample_directory_path, filenames[i], columns_of_frequencies))
    #print(i)

  return pd.concat(list_df)

In [ ]:
# large then needed data so that slows this down but still needs work
df = pca_get_all_data(path_samples) 

In [ ]:
#count file name rows? to see what images are larger?
# filenames size is 3650
df #why is there 415959 rows? aren't the sizes supposed to be 10x10

In [ ]:
print(df.iloc[0])
print(df.iloc[99])

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Drop non-numeric columns like 'X', 'Y', and 'File'
features = df.drop(columns=['X', 'Y', 'File'])

# Standardize the data (if features have different scales)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Convert the NumPy array to a pandas DataFrame and preserve the original index
features_scaled_df = pd.DataFrame(features_scaled, index=df.index)

In [ ]:
# Drop rows with any NaN values, while preserving the index
features_scaled_dropped = features_scaled_df.dropna()

# Ensure that there are no more NaN values
print(features_scaled_dropped.isnull().sum())

# Verify the row count of features_scaled_dropped and df
print(f"features_scaled_dropped rows: {len(features_scaled_dropped)}")
print(f"Original df rows: {len(df)}")

# Initialize PCA and reduce to 2 principal components
pca = PCA(n_components=2)  # Adjust the number of components if needed
principal_components = pca.fit_transform(features_scaled_dropped)

# Create a DataFrame for the PCA results
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

# Check the index length after dropping NaNs
print(f"pca_df rows: {len(pca_df)}")

# Reset the index of df if it has a custom index
df_reset = df.reset_index(drop=True)

# Align the rows of df with the rows that remain in features_scaled_dropped
pca_df['X'] = df_reset.loc[features_scaled_dropped.index, 'X'].values
pca_df['Y'] = df_reset.loc[features_scaled_dropped.index, 'Y'].values
pca_df['File'] = df_reset.loc[features_scaled_dropped.index, 'File'].values

# Show the first few rows of the PCA result
print(pca_df.head())

In [ ]:
# Visualize the first two principal components
plt.figure(figsize=(8, 6))
plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['X'] + pca_df['Y'], cmap='viridis')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA: First Two Principal Components')
plt.colorbar(label='Pixel Position (X + Y)')
plt.show()

In [ ]:
# Explained variance ratio
print(f'Explained variance ratio for each component: {pca.explained_variance_ratio_}')

In [ ]:
# Determine the number of components to explain 90% of the variance
pca = PCA(n_components=0.90)  # Keep enough components to explain 90% variance
principal_components = pca.fit_transform(features_scaled_dropped)
explained_variance_ratio = pca.explained_variance_ratio_

pca_full = PCA()
principal_components_full = pca_full.fit_transform(features_scaled_dropped)
explained_variance_ratio_full = pca_full.explained_variance_ratio_

print(f"Number of components to explain 90% variance: {pca.n_components_}")

In [ ]:
pca_full = PCA()
principal_components_full = pca_full.fit_transform(features_scaled_dropped)
explained_variance_ratio_full = pca_full.explained_variance_ratio_
pca_full.explained_variance_ratio_[:10]

In [ ]:
# Plot the explained variance ratio for all components
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(explained_variance_ratio_full) + 1), explained_variance_ratio_full, marker='o', linestyle='--')
plt.title('Explained Variance per Principal Component')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.xticks(range(1, len(explained_variance_ratio_full) + 1))
plt.grid(True)
plt.show()

# Determine the number of components to explain 90% of the variance
pca = PCA(n_components=0.90)
principal_components = pca.fit_transform(features_scaled_dropped)
explained_variance_ratio = pca.explained_variance_ratio_

print(f"Number of components to explain 90% variance: {pca.n_components_}")

# Optional: Cumulative explained variance plot
cumulative_variance = np.cumsum(explained_variance_ratio_full)
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--')
plt.axhline(y=0.90, color='r', linestyle='--', label="90% Variance")
plt.title('Cumulative Explained Variance')
plt.xlabel('Principal Componenst')
plt.ylabel('Cumulative Explained Variance')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Access the component loadings (how much each feature contributes to each component)
component_loadings = pca.components_

# Convert it to a DataFrame for easier inspection
loadings_df = pd.DataFrame(component_loadings, columns=features.columns)

# For each component, get the top 6 features with the highest absolute loadings
top_n = pca.n_components_
top_features = {}

for i in range(loadings_df.shape[0]):  # Loop through each component
    # Sort the values by absolute magnitude and get the top 6
    sorted_loadings = loadings_df.iloc[i].abs().sort_values(ascending=False).head(top_n)
    top_features[f"Component {i+1}"] = sorted_loadings

# Convert the result to a DataFrame for better presentation
top_features_df = pd.DataFrame(top_features)

# Print the top 6 features for each component
print(top_features_df)

In [ ]:
# Create a set to store the distinct frequencies used in the top features
used_frequencies = set()

# Loop through each component and add the top N frequencies to the set
for i in range(loadings_df.shape[0]):  # Loop through each component
    # Get the top features for the component by sorting by absolute magnitude
    sorted_loadings = loadings_df.iloc[i].abs().sort_values(ascending=False).head(top_n)
    # Add the indices (frequencies) of these top features to the set
    used_frequencies.update(sorted_loadings.index)

# Count the total number of unique frequencies used
total_used_frequencies = len(used_frequencies)

# Print the total number of unique frequencies
print(f"Total number of frequencies used across all components: {total_used_frequencies}")

# Print the names of the features (frequencies) used
print("\nFrequencies used across all components:")
for feature in used_frequencies:
    print(feature)

### Add Principal Components To Data

In [ ]:
# Convert the principal components to a DataFrame
principal_components_df = pd.DataFrame(
    principal_components, 
    columns=[f'PC{i+1}' for i in range(pca.n_components_)]
)

# Reset the index for consistency
principal_components_df = principal_components_df.reset_index(drop=True)

# Reset index in df to align with features_scaled_dropped
df_reset = df.reset_index(drop=True)

# Select 'X', 'Y', and 'File' columns from df
xy_file_df = df_reset[['X', 'Y', 'File']].iloc[:len(principal_components_df)].reset_index(drop=True)

# Concatenate the selected columns with the principal components
data_with_pcs = pd.concat([xy_file_df, principal_components_df], axis=1)

# Print the first 100 rows of the new dataset to verify
print(data_with_pcs.head())
#use the data_with_pcs to train the model

### Visualize an image

In [ ]:
# Step 1: Extract the first 100 pixels, each with 3 components
# Assuming data_with_pcs is a DataFrame with columns PC1, PC2, PC3 for each pixel
first_100_pixels = data_with_pcs[['PC1', 'PC2', 'PC3']].iloc[:100].values  # First 100 pixels with 3 components each

# Step 2: Reshape the data to 10x10x3 (image_dim x image_dim x 3 components per pixel)
image_dim = 10  # 10x10 image
reconstructed_image = first_100_pixels.reshape(image_dim, image_dim, 3)  # 3 components per pixel

# Step 3: Combine the 3 components into a single grayscale image (for simplicity)
combined_image = np.mean(reconstructed_image, axis=-1)  # Averaging the 3 components for grayscale

# Step 4: Display the reconstructed image
plt.imshow(combined_image)  # Display as grayscale
plt.title('Reconstructed Image from PCA')
plt.axis('off')
plt.show()

# Transfer Learning With ResNet50 (Not Yet Functional)

In [ ]:
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.applications import ResNet50
#from tensorflow.keras.models import Model
#from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
#from tensorflow.keras.optimizers import Adam

In [ ]:
# 1. Load the ResNet50 model pre-trained on ImageNet without the top layer
#base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# 2. Add custom layers for terrain classification
#x = base_model.output
#x = GlobalAveragePooling2D()(x)  # Global Average Pooling
#x = Dense(1024, activation='relu')(x)  # Fully connected layer
#predictions = Dense(5, activation='softmax')(x)  # Output layer (assuming 5 terrain classes)

In [ ]:
# 3. Create the final model
#model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# 4. Freeze the layers of the base model (ResNet50) initially
#for layer in base_model.layers:
#    layer.trainable = False

In [ ]:
# 5. Compile the model
#model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 6. Prepare the dataset

In [ ]:
# 7. Train the model (initial training)
#model.fit(train_generator, epochs=5, steps_per_epoch=train_generator.samples // train_generator.batch_size)

In [ ]:
# 8. Fine-tune the model: Unfreeze the top layers of the base model
#for layer in base_model.layers[-10:]:  # Unfreeze the last 10 layers of ResNet50
#    layer.trainable = True


In [ ]:
# 9. Recompile and retrain the model
#model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
#model.fit(train_generator, epochs=5, steps_per_epoch=train_generator.samples // train_generator.batch_size)


In [ ]:
# 10. Save the model
#model.save('resnet_model.h5')

# Custom CNN

In [ ]:
%pip install openpyxl

In [ ]:
import pandas as pd

# Load the Excel file
excel_file = 'Labels/CNN_Sample_Boxes_Subset_241018.xlsx'

# Read the Excel file into a DataFrame
df = pd.read_excel(excel_file, sheet_name=5)

# Display the first few rows
print(df.head())

# Access specific columns
image_numbers = df['Sample_num']
labels = df['Class']

# Access specific rows
first_row = df.iloc[0]  # First row as a Series
first_value = df.iloc[0, 0]  # First cell value

# Iterate over rows
#for index, row in df.iterrows():
   # print(f"Sample_num: {row['Sample_num']}, Label: {row['Class']}")


In [ ]:
import os
import shutil
import pandas as pd

def assign_images_to_samples(image_folder, labels_excel, output_folder, sheet_index=5):
    """
    Assigns images to each Sample_num and Class pair from the DataFrame,
    matching the filenames exactly to the format Sample_num + "_".

    Args:
    - image_folder (str): Path to the folder containing the images.
    - labels_excel (str): Path to the Excel file containing Sample_num and Class labels.
    - output_folder (str): Path to the output folder where organized data will be saved.
    - sheet_index (int): Index of the sheet to read from the Excel file.

    Returns:
    - None
    """
    # Read the specified sheet from the Excel file
    df = pd.read_excel(labels_excel, sheet_name=sheet_index)

    # Convert Sample_num to integers, then to strings
    df['Sample_num'] = df['Sample_num'].apply(lambda x: str(int(x)) if not pd.isna(x) else None)

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through the DataFrame
    for index, row in df.iterrows():
        sample_num = row['Sample_num']
        label = row['Class']

        if sample_num is None:
            print(f"Skipping row {index} with missing Sample_num")
            continue

        # Find the matching image in the image folder
        assigned_image = None
        for image_file in os.listdir(image_folder):
            # Match files that start with Sample_num followed by "_"
            if image_file.startswith(f"{sample_num}_") and image_file.lower().endswith(('.tif', '.jpg', '.jpeg', '.png')):
                assigned_image = image_file
                break

        # Check if an image was found
        if assigned_image:
            # Create a subdirectory for the label if it doesn't exist
            label_folder = os.path.join(output_folder, str(label))
            os.makedirs(label_folder, exist_ok=True)

            # Copy the image to the appropriate label folder
            source_path = os.path.join(image_folder, assigned_image)
            destination_path = os.path.join(label_folder, assigned_image)
            shutil.copy(source_path, destination_path)
        else:
            print(f"No image found for Sample_num {sample_num}")

    print(f"Images organized into {output_folder}.")

# Example usage
#assign_images_to_samples(
#    image_folder="/content/drive/MyDrive/Landcover-Classification_11-17/Images",
#    labels_excel="/content/drive/MyDrive/Landcover-Classification_11-17/CNN_Sample_Boxes_Subset_241018.xlsx",
#    output_folder="/content/drive/MyDrive/Landcover-Classification_11-17/Organized_Images"
#)


In [ ]:
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import numpy as np


# Step 1: Prepare the input and labels
# Assuming the data has a target class column. For simplicity, we'll create dummy labels.
# Replace 'dummy_labels' with your actual labels if available.
num_samples = len(data_with_pcs)
#print(num_samples)
dummy_labels = np.random.randint(0, 2, size=num_samples)  # Replace with actual labels

# Extract the first three principal components as features
X = data_with_pcs[['PC1', 'PC2', 'PC3']].values
y = LabelEncoder().fit_transform(dummy_labels)  # Encode labels if they are categorical

#print(f"Original shape of X: {X.shape}")
#print(f"Total elements in X: {X.size}")


# Determine number of valid samples
num_samples = X.shape[0] // 100  # Each sample needs 10x10=100 rows

# Truncate and reshape
X = X[:num_samples * 100]        # Truncate extra rows
X = X.reshape(num_samples, 10, 10, 3)  # Reshape to (num_samples, 10, 10, 3)

# Adjust labels
y = y[:num_samples]

#print(f"New shape of X: {X.shape}")  # Should be (num_samples, 10, 10, 3)
#print(f"New length of y: {len(y)}")

# Proceed with train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y[:num_samples], test_size=0.2, random_state=42)


# Step 2: Define a simple CNN
model = models.Sequential([
    layers.Input(shape=(10, 10, 3)),
    
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Step 3: Train the model ###This method might take longer to run but provides a high accuracy
kf = KFold(n_splits=5)
for train_idx, val_idx in kf.split(X):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20)



print("\n")
# Step 4: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

print("\n")
# Step 5: Use the model to predict
predictions = model.predict(X_test)
print(f"Predictions: {np.argmax(predictions, axis=1)[:10]}")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = np.argmax(model.predict(X_test), axis=1)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
from tensorflow.keras import models, layers, regularizers

def hyperspectral_cnn(input_shape, num_classes):
    """
    Create a 2D CNN model for hyperspectral image classifacation with reduced overfitting.
    """
    model = models.Sequential([
        # First Convolutional Block
        layers.Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape, kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Conv2D(16, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),
        
        # Second Convolutional Block
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),
        
         # Third Convolutional Block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),
        
        # Fourth Convolutional Block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.5),
        
        # Dense Layer
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        # Output Layer
        layers.Dense(num_classes, activation='softmax')
    ])
    
    # Compile model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
from tensorflow.keras import models, layers, regularizers

def new_hyperspectral_cnn(input_shape, num_classes):
    """
    Create a 2D CNN model optimized for small spatial dimensions (e.g., 10x10).
    Handles hyperspectral image classification with reduced spatial pooling.
    """
    model = models.Sequential([
        # First Convolutional Block
        layers.Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape, kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Conv2D(16, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), padding='same'),  # Spatial dims: 10x10 -> 5x5
        layers.Dropout(0.3),
        
        # Second Convolutional Block
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), padding='same'),  # Spatial dims: 5x5 -> 3x3
        layers.Dropout(0.3),
        
        # Third Convolutional Block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        # Replace MaxPooling with GlobalAveragePooling to handle small dimensions
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.4),
        
        # Dense Layer
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        # Output Layer
        layers.Dense(num_classes, activation='softmax')
    ])
    
    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
%pip install tifffile
%pip install imagecodecs

In [ ]:
from tifffile import imread

def get_images_and_labels_tifffile(image_folder, labels_file):
    """
    Reads TIFF images and their corresponding labels from a folder and CSV file.
    Skips images without corresponding labels in the CSV file.

    Parameters:
    - image_folder: str, path to the folder containing the images.
    - labels_file: str, path to the CSV file containing the labels.

    Returns:
    - X: List of image data as NumPy arrays.
    - y: List of corresponding labels.
    """
    # Read the labels CSV
    labels_df = pd.read_csv(labels_file)

    # Create a dictionary mapping Sample_num to Class
    labels_dict = dict(zip(labels_df['Sample_num'].astype(str), labels_df['Class']))

    X = []
    y = []

    # Get all TIFF image files
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.tif')]

    for image_file in image_files:
        # Extract the sample number from the filename
        sample_num = image_file.split('_')[0]

        # Skip images without labels
        if sample_num not in labels_dict:
            print(f"No label found for image: {image_file}. Skipping.")
            continue

        image_path = os.path.join(image_folder, image_file)

        try:
            # Read the TIFF image using tifffile
            image = imread(image_path)

            # Normalize image values to [0, 1]
            image = image.astype(np.float32) / 255.0

            # Append the image and its label to the lists
            X.append(image)
            y.append(labels_dict[sample_num])
        except Exception as e:
            print(f"Error reading {image_file}: {e}")

    return X, y

In [ ]:
def get_images_and_labels_rasterio(image_folder, labels_file):
    """
    Reads TIFF images using rasterio and their corresponding labels from a folder and CSV file.
    Skips images without corresponding labels in the CSV file.

    Parameters:
    - image_folder: str, path to the folder containing the images.
    - labels_file: str, path to the CSV file containing the labels.

    Returns:
    - X: List of image data as NumPy arrays.
    - y: List of corresponding labels.
    """
    # Read the labels CSV
    labels_df = pd.read_csv(labels_file)

    # Create a dictionary mapping Sample_num to Class
    labels_dict = dict(zip(labels_df['Sample_num'].astype(str), labels_df['Class']))

    X = []
    y = []

    # Get all TIFF image files
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.tif')]

    for image_file in image_files:
        # Extract the sample number from the filename
        sample_num = image_file.split('_')[0]

        # Skip images without labels
        if sample_num not in labels_dict:
            print(f"No label found for image: {image_file}. Skipping.")
            continue

        image_path = os.path.join(image_folder, image_file)

        try:
            # Read the TIFF image using rasterio
            with rasterio.open(image_path) as src:
                image = src.read()  # Read the image as a NumPy array (bands x rows x cols)

            # Normalize image values to [0, 1]
            image = image.astype(np.float32) / np.iinfo(src.dtypes[0]).max  # Normalize by the max possible value

            # Append the image and its label to the lists
            X.append(image)
            y.append(labels_dict[sample_num])
        except Exception as e:
            print(f"Error reading {image_file}: {e}")

    return X, y

In [ ]:
def process_labels_and_save(csv_file_path, labels_to_remove):
    """
    Processes the class labels in a CSV file:
    - Converts all class labels to lowercase for consistency.
    - Removes rows with specific labels and rows with NaN values.
    - Saves a new CSV file in the same directory as the original file.

    Parameters:
    - csv_file_path: str, path to the CSV file.
    - labels_to_remove: list, class labels to remove (not case-sensitive).

    Returns:
    - processed_df: pandas DataFrame with updated labels.
    - new_file_path: str, path to the saved new CSV file.
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)

    # Ensure the 'Class' column exists
    if 'Class' not in df.columns:
        raise ValueError("The CSV file must have a 'Class' column.")

    # Convert all class labels to lowercase
    df['Class'] = df['Class'].str.lower()

    # Remove rows with NaN values in the 'Class' column
    df = df.dropna(subset=['Class'])

    # Remove rows with labels to remove (case-insensitive)
    labels_to_remove_lower = [label.lower() for label in labels_to_remove]
    df = df[~df['Class'].isin(labels_to_remove_lower)]

    # Save the processed DataFrame to a new CSV file in the same directory
    base_dir = os.path.dirname(csv_file_path)
    new_file_name = "Processed_" + os.path.basename(csv_file_path)
    new_file_path = os.path.join(base_dir, new_file_name)
    df.to_csv(new_file_path, index=False)

    return df, new_file_path

In [ ]:
def augment_images(X, y):
    """
    Augments images stored as NumPy arrays and updates labels accordingly.

    Parameters:
    - X: List of original images as NumPy arrays.
    - y: List of original labels.

    Returns:
    - X_augmented: List of original and augmented images as NumPy arrays.
    - y_augmented: List of labels corresponding to X_augmented.
    """
    X_augmented = X.copy()
    y_augmented = y.copy()

    for i in range(len(X)):
        image = X[i]
        label = y[i]

        # Flip vertically
        flipped_vert = np.flipud(image)
        X_augmented.append(flipped_vert)
        y_augmented.append(label)

        # Flip horizontally
        flipped_horiz = np.fliplr(image)
        X_augmented.append(flipped_horiz)
        y_augmented.append(label)

        # Rotate 90 degrees clockwise
        rotated = np.rot90(image, k=-1)  
        X_augmented.append(rotated)
        y_augmented.append(label)
        
        rotated_counterclockwise = np.rot90(image, k=1)  
        X_augmented.append(rotated_counterclockwise)
        y_augmented.append(label)

    return X_augmented, y_augmented

In [ ]:
from sklearn.model_selection import train_test_split
import cv2


def prepare_data_for_training(X, y, test_size=0.2, random_state=42):
    """
    Prepares data for training by resizing and normalizing images and splitting into train/test sets.

    Parameters:
    - X: List of images as NumPy arrays.
    - y: List of labels.
    - test_size: Fraction of the dataset to reserve for testing.
    - random_state: Seed for random number generator.

    Returns:
    - X_train: Training images as NumPy arrays.
    - X_test: Testing images as NumPy arrays.
    - y_train: Training labels.
    - y_test: Testing labels.
    """
    

    # Define target image size
    target_size = (10,10)

    # Initialize lists to hold resized image arrays
    X_resized = []

    for img in X:
        # Resize image using OpenCV
        resized_img = cv2.resize(img, target_size, interpolation=cv2.INTER_LINEAR)
        X_resized.append(resized_img)

    # Normalize pixel values to [0, 1]
    X_resized = np.array(X_resized).astype(np.float32) / 255.0
    y_array = np.array(y)

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X_resized, y_array,
        test_size=test_size,
        random_state=random_state,
        stratify=y_array
    )

    return X_train, X_test, y_train, y_test

In [ ]:
from scipy.ndimage import zoom

def clean_data(X, y, target_shape=(10, 10)):
    """
    Cleans and preprocesses the data:
    - Removes invalid samples (nan or inf values in X).
    - Resizes images to the target shape.

    Parameters:
    - X: list of image data (e.g., NumPy arrays of varying shapes).
    - y: list of corresponding labels.
    - target_shape: tuple, desired (height, width) for resizing images.

    Returns:
    - X_cleaned: numpy.ndarray, cleaned and resized input data.
    - y_cleaned: numpy.ndarray, cleaned labels.
    """
    X_cleaned = []
    y_cleaned = []

    for i, img in enumerate(X):
        # Check for valid images (no nan or inf values)
        if img is not None and not np.isnan(img).any() and not np.isinf(img).any():
            try:
                # Resize image to target shape while preserving channels
                current_shape = img.shape
                if current_shape[:2] != target_shape:
                    zoom_factors = (target_shape[0] / current_shape[0],
                                    target_shape[1] / current_shape[1],
                                    1)  # Keep channels unchanged
                    img = zoom(img, zoom_factors, order=1)  # Bilinear interpolation

                # Append the resized and valid image
                X_cleaned.append(img)
                y_cleaned.append(y[i])
            except AttributeError:
                print(f"Image {i} is not a valid NumPy array. Skipping.")
        else:
            print(f"Image {i} contains invalid values (nan or inf). Skipping.")

    return np.array(X_cleaned), np.array(y_cleaned)

In [ ]:
imageFolderPath = 'Data/Images'
labelsFilePath = 'Data/ConsolidatedLabels.csv'

df, new_file_path = process_labels_and_save(labelsFilePath, ['nan', 'Tie', '#N/A'])

# rasterio_X, rasterio_y = get_images_and_labels_rasterio(imageFolderPath, new_file_path)
tifffile_X, tifffile_y = get_images_and_labels_tifffile(imageFolderPath, new_file_path)

# cleaned_tifffile_X, cleaned_tifffile_y = clean_data(tifffile_X, tifffile_y)

# new_rasterio_X, new_rasterio_y = augment_images(rasterio_X, rasterio_y)
new_tifffile_X, new_tifffile_y = augment_images(tifffile_X, tifffile_y)

# print(f"Original data: {len(rasterio_X)} images, {len(rasterio_y)} labels")
# print(f"Augmented data: {len(new_rasterio_X)} images, {len(new_rasterio_y)} labels")

print(f"Original data: {len(tifffile_X)} images, {len(tifffile_y)} labels")

print(f"Augmented data: {len(new_tifffile_X)} images, {len(new_tifffile_y)} labels")



In [ ]:

cleaned_tifffile_X, cleaned_tifffile_y = clean_data(new_tifffile_X, new_tifffile_y, (10, 10))
print(f"Cleaned data: {len(cleaned_tifffile_X)} images, {len(cleaned_tifffile_y)} labels")


In [ ]:

X_train, X_test, y_train, y_test = prepare_data_for_training(cleaned_tifffile_X, cleaned_tifffile_y)



In [ ]:
# Create a mapping from label strings to integers
unique_labels = sorted(set(y_train))
label_to_int = {label: idx for idx, label in enumerate(unique_labels)}

print(f"Label to integer mapping: {label_to_int}")
print(f"Unique labels: {unique_labels}")
print("unique labels:",set(unique_labels))
print("unique labels:",len(set(unique_labels)))

# Convert y_train and y_test to integers
y_train = [label_to_int[label] for label in y_train]
y_test = [label_to_int[label] for label in y_test]

In [ ]:
cnn_model = hyperspectral_cnn(input_shape=(dataset_1.height, dataset_1.width, dataset_1.count), num_classes=12)

In [ ]:
# Print shapes to check compatibility
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Model input shape:", cnn_model.input_shape)

# Print data types to ensure consistency
print("X_train dtype:", X_train.dtype)
print("X_test dtype:", X_test.dtype)
print("y_train dtype:", type(y_train))
print("y_test dtype:", type(y_test))

# Convert y_train and y_test to NumPy arrays with integer type
y_train = np.array(y_train, dtype=np.int32)
y_test = np.array(y_test, dtype=np.int32)

# Verify the conversion
print("y_train dtype:", y_train.dtype)
print("y_train shape:", y_train.shape)
print("y_test dtype:", y_test.dtype)
print("y_test shape:", y_test.shape)

# Convert X_train and X_test to NumPy arrays with float32 type
X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)

# Verify the conversion
print("X_train dtype:", X_train.dtype)
print("X_test dtype:", X_test.dtype)

In [ ]:
# Check unique labels and their range

# print("X_train range:", X_train.min(), X_train.max())

# print("Unique labels in y_train:", np.unique(y_train))
# print("Unique labels in y_test:", np.unique(y_test))



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6)


history = cnn_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping, lr_scheduler]
)

In [ ]:
def plot_training_history(history, cnn_model, X_test, y_test):
    """
    Plots the training and validation accuracy and loss from the history object.
    """
    plt.figure(figsize=(14, 5))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.grid()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid()

    plt.tight_layout()
    
    y_pred = cnn_model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Compute confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred_classes)

    # Display the confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=range(12))
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy vs val_accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Validation Accuracy')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Plot loss vs val_loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Validation Loss')
plt.legend()
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Get predicted labels
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=range(12))
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Get the highest validation accuracy and corresponding epoch
best_val_accuracy = max(history.history['val_accuracy'])
best_epoch = history.history['val_accuracy'].index(best_val_accuracy) + 1

print(f"Highest Validation Accuracy: {best_val_accuracy:.4f}")
print(f"Epoch with Highest Validation Accuracy: {best_epoch}")

# Get predicted probabilities
y_pred = cnn_model.predict(X_test)

# Convert to percentages
y_pred_percentages = y_pred * 100

for i in range(best_epoch):
    print(f"Sample {i}:")
    for cls, prob in enumerate(y_pred_percentages[i]):
        print(f"  Class {cls}: {prob:.2f}%")

In [ ]:
from collections import Counter

def plot_class_distribution(y, class_names, title="Class Distribution"):
    """
    Plots a histogram showing the distribution of samples across classes.

    Parameters:
    - y: array-like, labels for the dataset (as strings or integers).
    - class_names: list, names of the classes corresponding to the labels.
    - title: str, title for the plot.

    Returns:
    - None
    """
    # Count occurrences of each class
    class_counts = Counter(y)
    
    # Ensure the order of bars matches class_names
    counts = [class_counts.get(cls, 0) for cls in class_names]

    # Create a bar plot
    plt.figure(figsize=(10, 6))  # Adjust figure size as needed
    plt.bar(class_names, counts)
    plt.xlabel('Class')
    plt.ylabel('Number of Samples')
    plt.title(title)
    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
    plt.grid(axis='y')
    plt.tight_layout()  # Adjust layout to prevent label cutoff
    plt.show()
    

def get_class_counts(y, class_names=None):
    """
    Returns the count of samples for each class in the dataset.

    Parameters:
    - y: array-like, labels for the dataset (as strings or integers).
    - class_names: list, optional, names of the classes corresponding to the labels.

    Returns:
    - class_counts: dict, mapping of class names (or labels) to their counts.
    """
    # Count occurrences of each class
    class_counts = Counter(y)
    
    # Map counts to class names if provided
    if class_names is not None:
        class_counts = {class_names[int(label)]: count for label, count in class_counts.items()}

    return class_counts

# Assuming y_train and y_test are NumPy arrays or lists of numeric or string labels
combined_labels = np.concatenate((y_train, y_test))  # Combine datasets if needed
combined_labels = [str(label) for label in combined_labels]  # Convert to strings if necessary

# Define class names (if numeric labels are used)
class_names = sorted(set(combined_labels))  # Replace with your actual class names if needed

# Get class counts
class_counts = get_class_counts(combined_labels)

# Print class counts
for class_name, count in class_counts.items():
    print(f"{class_name}: {count}")

# Plot the class distribution
plot_class_distribution(combined_labels, class_names, title="Class Distribution for Entire Dataset")

In [ ]:
"""Add class weights to help with data imbalance"""
from sklearn.utils.class_weight import compute_class_weight

def calculate_class_weights(class_counts):
    """
    Calculates class weights to handle class imbalance.

    Parameters:
    - class_counts: dict, mapping of class names or indices to their counts.

    Returns:
    - class_weights: dict, mapping of class indices to weights.
    """
    total_samples = sum(class_counts.values())
    num_classes = len(class_counts)

    # Ensure class indices are numeric
    class_weights = {
        idx: total_samples / (num_classes * count)
        for idx, count in enumerate(class_counts.values())
    }

    return class_weights

# Example usage
# Combine training and testing labels
combined_labels = np.concatenate((y_train, y_test))  # Ensure y_train and y_test are NumPy arrays
combined_labels = [str(label) for label in combined_labels]  # Convert labels to strings if needed

# Dynamically get class counts
class_counts = get_class_counts(combined_labels)

# Calculate class weights
class_weights = calculate_class_weights(class_counts)

# Print class weights for verification
print("Class Weights:")
for idx, (class_name, weight) in enumerate(zip(class_counts.keys(), class_weights.values())):
    print(f"Class {idx} ('{class_name}') weight: {weight:.4f}")

In [ ]:
# Calculate class weights
class_weights = calculate_class_weights(class_counts)

cnn_model_2 = hyperspectral_cnn(input_shape=(10, 10, 373), num_classes=12)

# Train the model with class weights
history_2 = cnn_model_2.fit(
    X_train, 
    y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    class_weight=class_weights,
    callbacks=[early_stopping, lr_scheduler]
)

In [ ]:
# Plot accuracy vs val_accuracy
plt.plot(history_2.history['accuracy'], label='Training Accuracy')
plt.plot(history_2.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Validation Accuracy')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Plot loss vs val_loss
plt.plot(history_2.history['loss'], label='Training Loss')
plt.plot(history_2.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Validation Loss')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# # Get predicted labels
# y_pred_2 = cnn_model_2.predict(X_test)
# y_pred_classes_2 = np.argmax(y_pred_2, axis=1)

# # Compute confusion matrix
# conf_matrix_2 = confusion_matrix(y_test, y_pred_classes_2)

# # Display the confusion matrix
# disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix_2, display_labels=range(12)) 
# plt.title('Confusion Matrix')
# plt.show()


# Get predicted labels
y_pred_2 = cnn_model_2.predict(X_test)
y_pred_classes_2 = np.argmax(y_pred_2, axis=1)

# Compute confusion matrix
conf_matrix_2 = confusion_matrix(y_test, y_pred_classes_2)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix_2, display_labels=range(12))
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Get the highest validation accuracy and corresponding epoch
best_val_accuracy_2 = max(history_2.history['val_accuracy'])
best_epoch_2 = history_2.history['val_accuracy'].index(best_val_accuracy_2) + 1

print(f"Highest Validation Accuracy: {best_val_accuracy_2:.4f}")
print(f"Epoch with Highest Validation Accuracy: {best_epoch_2}")

# Get predicted probabilities
y_pred_2 = cnn_model_2.predict(X_test)

# Convert to percentages
y_pred_percentages_2 = y_pred_2 * 100

for i in range(best_epoch_2):
    print(f"Sample {i}:")
    for cls, prob in enumerate(y_pred_percentages_2[i]):
        print(f"  Class {cls}: {prob:.2f}%")

In [ ]:
cnn_model.summary()


In [ ]:
cnn_model_2.summary()

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten
# from tensorflow.keras.models import Model

# def modify_pretrained_model(new_input_shape, base_model_name='ResNet50', num_classes=10):
#     """
#     Modifies a pre-trained model to accept a new input shape and updates the first layer accordingly.

#     Args:
#         new_input_shape (tuple): The new input shape, e.g., (10, 10, 373).
#         base_model_name (str): Name of the pre-trained model to use. Default is 'ResNet50'.
#         num_classes (int): Number of classes for the final classification layer.

#     Returns:
#         tf.keras.Model: The modified model ready for training.
#     """
#     # Validate input shape for compatibility with pre-trained models
#     if len(new_input_shape) != 3:
#         raise ValueError("Input shape must be a tuple of (height, width, channels).")

#     # Load the pre-trained model
#     if base_model_name == 'ResNet50':
#         base_model = tf.keras.applications.ResNet50(
#             include_top=False,  # Exclude top layers
#             weights='imagenet',  # Use ImageNet pre-trained weights
#             input_shape=(None, None, 3)  # Keep original input channels for weight loading
#         )
#     else:
#         raise ValueError(f"Unsupported base model: {base_model_name}")
    
#     # Create a new input layer
#     new_input = Input(shape=new_input_shape)

#     # Project 373 input channels to 3 using a 1x1 convolution
#     x = Conv2D(3, (1, 1), activation='linear', name='channel_projection')(new_input)

#     # Connect the rest of the pre-trained model
#     x = base_model(x)

#     # Add custom classification layers
#     x = Flatten()(x)
#     x = Dense(256, activation='relu')(x)
#     output = Dense(num_classes, activation='softmax')(x)

#     # Create the new model
#     model = Model(inputs=new_input, outputs=output)

#     # Compile the model
#     model.compile(optimizer='adam', 
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
#               metrics=['accuracy'])

#     return model

# # Example usage
# new_input_shape = (10, 10, 373)
# num_classes = 12
# new_model = modify_pretrained_model(new_input_shape, num_classes=num_classes)
# # new_model.summary()

# new_history = new_model.fit(
#     X_train, 
#     y_train,
#     validation_data=(X_test, y_test),
#     epochs=50,
#     batch_size=32,
#     class_weight=class_weights,
#     callbacks=[early_stopping, lr_scheduler]
# )

# Feed Data To HybridSN

#### This may be problematic. HybridSN does NOT provide the pre-trained model. Instead we'd have to train it ourselves, which is providing difficult due to compatibility issues

### Retrieve The Model

In [ ]:
# !git clone https://github.com/gokriznastic/HybridSN.git

# Feed Data To Custom Model